In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.stats import entropy
import json
import os
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Function to load and combine large JSON files
def load_data(file_paths):
    dfs = []
    transaction_types = ['deposits', 'borrows', 'repays', 'withdrawals', 'liquidations']

    for file in file_paths:
        logging.info(f"Loading {file}...")
        try:
            with open(file, 'r') as f:
                data = json.load(f)
        except Exception as e:
            logging.error(f"Failed to load {file}: {e}")
            continue

        # Process each transaction type
        for tx_type in transaction_types:
            if tx_type in data and data[tx_type]:
                try:
                    # Normalize nested fields
                    tx_data = pd.json_normalize(data[tx_type])
                    # Rename columns for consistency
                    tx_data = tx_data.rename(columns={
                        'account.id': 'wallet_address',
                        'asset.symbol': 'asset',
                        'amount': 'amount',
                        'timestamp': 'timestamp'
                    })
                    # Check required columns
                    required_cols = ['wallet_address', 'amount', 'timestamp']
                    if not all(col in tx_data.columns for col in required_cols):
                        logging.warning(f"Missing required columns in {tx_type} from {file}")
                        continue
                    # Add transaction type column
                    tx_data['transaction_type'] = tx_type[:-1]  # Remove 's' (e.g., 'deposits' -> 'deposit')
                    # Select relevant columns
                    tx_data = tx_data[['wallet_address', 'transaction_type', 'amount', 'timestamp', 'asset']]
                    # Convert amount to numeric
                    tx_data['amount'] = pd.to_numeric(tx_data['amount'], errors='coerce')
                    dfs.append(tx_data)
                except Exception as e:
                    logging.error(f"Error processing {tx_type} in {file}: {e}")
                    continue

    # Combine all transactions
    if dfs:
        return pd.concat(dfs, ignore_index=True)
    else:
        raise ValueError("No valid transaction data found in the provided files.")

# Feature engineering
def engineer_features(df):
    try:
        # Convert timestamp to datetime
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

        # Aggregate by wallet
        wallet_features = df.groupby('wallet_address').agg({
            'amount': [
                ('total_deposit', lambda x: x[df['transaction_type'] == 'deposit'].sum()),
                ('total_borrow', lambda x: x[df['transaction_type'] == 'borrow'].sum()),
                ('total_repay', lambda x: x[df['transaction_type'] == 'repay'].sum()),
                ('total_withdraw', lambda x: x[df['transaction_type'] == 'withdraw'].sum()),
                ('total_liquidation', lambda x: x[df['transaction_type'] == 'liquidation'].sum())
            ],
            'transaction_type': [
                ('count_deposit', lambda x: (x == 'deposit').sum()),
                ('count_borrow', lambda x: (x == 'borrow').sum()),
                ('count_repay', lambda x: (x == 'repay').sum()),
                ('count_withdraw', lambda x: (x == 'withdraw').sum()),
                ('count_liquidation', lambda x: (x == 'liquidation').sum())
            ],
            'timestamp': [
                ('first_tx', 'min'),
                ('last_tx', 'max'),
                ('tx_count', 'count')
            ]
        })

        # Flatten column names
        wallet_features.columns = [f"{x}_{y}" for x, y in wallet_features.columns]

        # Calculate additional features
        wallet_features['borrow_to_deposit_ratio'] = (
            wallet_features['amount_total_borrow'] / (wallet_features['amount_total_deposit'] + 1e-6)
        )
        wallet_features['repay_to_borrow_ratio'] = (
            wallet_features['amount_total_repay'] / (wallet_features['amount_total_borrow'] + 1e-6)
        )
        wallet_features['liquidation_to_borrow_ratio'] = (
            wallet_features['amount_total_liquidation'] / (wallet_features['amount_total_borrow'] + 1e-6)
        )
        wallet_features['activity_span_days'] = (
            (wallet_features['timestamp_last_tx'] - wallet_features['timestamp_first_tx']).dt.days
        )
        wallet_features['avg_tx_per_day'] = (
            wallet_features['timestamp_tx_count'] / (wallet_features['activity_span_days'] + 1)
        )

        # Transaction pattern entropy
        def calculate_entropy(wallet_df):
            times = wallet_df['timestamp'].sort_values()
            intervals = times.diff().dt.total_seconds().dropna()
            if len(intervals) < 2:
                return 0
            hist, _ = np.histogram(intervals, bins=10, density=True)
            return entropy(hist + 1e-6)

        entropy_features = df.groupby('wallet_address').apply(calculate_entropy).rename('tx_entropy')
        wallet_features = wallet_features.join(entropy_features)

        # Handle infinities and NaNs
        wallet_features = wallet_features.replace([np.inf, -np.inf], 0).fillna(0)

        return wallet_features
    except Exception as e:
        logging.error(f"Error in feature engineering: {e}")
        raise

# Scoring model
def compute_scores(features):
    try:
        # Select relevant features for clustering
        feature_cols = [
            'amount_total_deposit', 'amount_total_borrow', 'amount_total_repay',
            'amount_total_withdraw', 'amount_total_liquidation',
            'borrow_to_deposit_ratio', 'repay_to_borrow_ratio',
            'liquidation_to_borrow_ratio', 'count_deposit', 'count_borrow',
            'count_repay', 'count_withdraw', 'count_liquidation',
            'activity_span_days', 'avg_tx_per_day', 'tx_entropy'
        ]

        X = features[feature_cols].values
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # K-means clustering
        kmeans = KMeans(n_clusters=5, random_state=42)
        features['cluster'] = kmeans.fit_predict(X_scaled)

        # Score based on cluster characteristics
        cluster_scores = []
        for cluster in range(5):
            cluster_data = features[features['cluster'] == cluster]
            score = (
                0.3 * cluster_data['repay_to_borrow_ratio'].mean() -
                0.3 * cluster_data['liquidation_to_borrow_ratio'].mean() +
                0.2 * cluster_data['amount_total_deposit'].mean() /
                (cluster_data['amount_total_borrow'].mean() + 1e-6) +
                0.1 * cluster_data['activity_span_days'].mean() /
                (cluster_data['avg_tx_per_day'].mean() + 1e-6) -
                0.1 * cluster_data['tx_entropy'].mean()
            )
            cluster_scores.append(score)

        # Normalize scores to 0–100
        cluster_scores = np.array(cluster_scores)
        normalized_scores = 100 * (cluster_scores - cluster_scores.min()) / (cluster_scores.max() - cluster_scores.min() + 1e-6)
        features['score'] = features['cluster'].map(dict(enumerate(normalized_scores)))

        return features[['score']]
    except Exception as e:
        logging.error(f"Error in scoring: {e}")
        raise

# Main execution
if __name__ == "__main__":
    # Placeholder for file paths (replace with actual paths from Google Drive)
    file_paths = [
        "/content/compoundV2_transactions_ethereum_chunk_0.json",
        "/content/compoundV2_transactions_ethereum_chunk_4.json",
        "/content/compoundV2_transactions_ethereum_chunk_7.json"
    ]

    try:
        # Load data
        df = load_data(file_paths)

        # Engineer features
        features = engineer_features(df)

        # Compute scores
        scores = compute_scores(features)

        # Output top 1,000 wallets
        top_wallets = scores.sort_values('score', ascending=False).head(1000)
        top_wallets.to_csv('wallet_scores.csv')

        logging.info("Scoring complete. Output saved to wallet_scores.csv")
    except Exception as e:
        logging.error(f"Main execution failed: {e}")